In [9]:
import csv
import pandas as pd
from nltk.corpus import stopwords
import re

<h1>Course Recommendation System</h1>

Data provided by Concordia Open Data, files used last updated May 30 2023

catalog is the dataset that contains the course description <br>
sr is the default dataset that we will join on to obtain the final "dataset"

In [10]:
# Use Sniffer to infer the delimiter of the CSV file
with open('CATALOG.csv', 'r') as file:
    dialect = csv.Sniffer().sniff(file.read(1024))
    delimiter = dialect.delimiter

# Read the CSV file using pandas with the inferred delimiter
catalog = pd.read_csv('CATALOG.csv', delimiter=delimiter)
catalog = catalog.drop([ "Website",'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23'], axis=1)
# Access the column containing potential commas

In [11]:
sr = pd.read_csv("CU_SR_OPEN_DATA_CATALOG.csv", encoding="utf_16_le")
sr = sr.drop(["Career"], axis=1)
# comb = pd.read_csv("CU_SR_OPEN_DATA_COMB_SECTIONS.csv", encoding="utf_16_le")

In [12]:
# catalog.columns
catalog["Subject_Catalog"] = catalog["Course code"] +"_"+ catalog["Course number"]

In [13]:
# sr.columns
sr["Subject_Catalog"] = sr["Subject"] +"_"+ sr["Catalog"]

In [14]:
dataset = sr.merge(catalog, on="Subject_Catalog")
dataset = dataset.drop(["Course code", "Course number", "Component Code", "Key", "Course ID", "Title"], axis=1)
dataset = dataset.fillna("")

<h1>Creation of Catalog Catagories </h1>

(ex: 200-level)

In [15]:
# returned as str instead of int for the sake of concatenation
def fun1(x):
    x = re.findall(r'\d+',x)[0]
    if int(x)<1000:
        return str((int(x)//100)*100)
    elif int(x)>=1000:
        return str((int(x)//1000)*10000)

dataset['Level'] = dataset['Catalog'].map(lambda x:fun1(x))

<h2> Categorical Breakdown

In [16]:
dataset["Degree"].value_counts()
dataset["Pre Requisite Description"].value_counts()
dataset["Equivalent Courses"].value_counts()

                        1605
ELEC 372 = ENGR 372        6
COEN 312 = COEN 212        6
COEN 421 = SOEN 422        5
AHSC538 = AHSC536          4
                        ... 
DFTT 211 = DFTT 298C       1
DFTT 211=SCEN 214          1
DFTT 341=SCEN 344          1
DFTT 470=SCEN 470          1
TESL471 = TESL498A         1
Name: Equivalent Courses, Length: 234, dtype: int64

<h2>Removing Stop Words & Encoding


In [17]:
enStop = stopwords.words('english')
frStop = stopwords.words('french')
dataset['Description_without_stopwords'] = dataset['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (enStop) and word not in (frStop)]))

In [20]:
''' 
Changes:
"Catalog" removed; "Level" added 
"Subject_Catalog" removed; "Subject" added
'''

dataset["Encoding"] = dataset["Long Title"]+ " " + dataset["Equivalent Courses"] + " " + dataset["Faculty"] + " " + dataset["Department"] + " " + dataset["Subject"] +" "+ dataset["Catalog"] +" "+ dataset["Level"] + " " + dataset["Degree"] + " " + dataset["Description_without_stopwords"]

dataset["Encoding"].to_csv("Encoding.csv")